In [1]:
import networkx as nx
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 

In [2]:
import multiprocessing
from multiprocessing import cpu_count

In [3]:
#################################################################################
############################### Helper Functions  ###############################
#################################################################################

def set_influence(G, value, node=None):
    '''
        Set influence of a node in a network G or
        set influence of all nodes to value.
        G      ::  a networkx graph
        node   ::  a reference to a node in G
        value  ::  an integer 0 or 1
    '''
    if node:
        G.nodes[node][label] = value
    else:
        influence_attrib = { i : 0 for i in range(N) }
        nx.set_node_attributes(G,influence_attrib, label)
        
def get_is_influenced(G, node):
    return G.nodes[node][label]
        
def get_number_influenced(G):
    '''
        Get the number of influenced nodes.
    '''
    return sum(nx.get_node_attributes(G, label).values())

def check_can_influence(G, node, q):
    '''
        Determines whether a node is influenced by
        its neighbours. (Threshold Check)
    '''
    if get_is_influenced(G, node) == 1:
        return False
    
    friends = list(G.neighbors(node))
    num_friends = len(friends)
    
    if num_friends == 0:
        return False
    
    num_influenced = sum([1 for friend in friends if G.nodes[friend][label] == 1])
    
    if num_influenced/num_friends > q:
        return True
    return False

In [4]:
#################################################################################
########################## Simulation Helper Functions ##########################
#################################################################################

def spread_influence(G, current_node, phi):
    '''
        Recursive function to spread influence 
        from current_node in G.
    '''
    
    ## Not interesting
    if get_is_influenced(G, current_node) == 0 and not check_can_influence(G, current_node, phi):
        return
    else:
        set_influence(G, 1, current_node)
        ## Find uninfluenced friends
        friends = list(G.neighbors(current_node))
        targets = [friend for friend in friends if G.nodes[friend][label] == 0]
        
        for friend in targets:
            spread_influence(G, friend, phi)
        
def get_vulnerable(G, phi):
    '''
        Check for vulnerable nodes.
        Return list of vulnerable nodes.
    '''
    vulnerable = []
    for node in G.nodes():
        if check_can_influence(G, node, phi):
            vulnerable.append(node)
    return vulnerable
        
def simulate_spread(G, nodes, phi):
    '''
        Simulates the spread of influence starting from each 
        node in nodes and returns a list containing the 
        number of influenced from starting at each node.
    '''
    S = []
    for node in nodes:
        G_tmp = G.copy()
        set_influence(G_tmp, 1, node)
        spread_influence(G_tmp, node, phi)
        
        vulnerable = get_vulnerable(G_tmp, phi)

        while len(vulnerable) != 0:
            for vul in vulnerable:
                spread_influence(G_tmp, vul, phi)
            vulnerable = get_vulnerable(G_tmp, phi)

        S.append(get_number_influenced(G_tmp))
        
    return S


In [5]:
%%time
#################################################################################
################################## Simulation ###################################
#################################################################################

N = 25
q = 0.1   
phi = 0.18
max_n_avg = 6
increment = 0.2
num_simulations = 50
label = 'is_influenced'

n_avg = np.arange(1, max_n_avg, increment)
p = [avg/(N-1) for avg in n_avg]
n = len(p)
S_influential, S_normal = np.zeros([num_simulations, n]), np.zeros([num_simulations, n])

for i in tqdm(range(num_simulations)):
    for j in range(n):
        probability = p[j]
        G = nx.erdos_renyi_graph(N, probability)
        set_influence(G, 0)
        ## Retrieve influential nodes - top q% and non-influential nodes
        degree_ordered_nodes = sorted(list(G.nodes()), key=lambda x: G.degree(x), reverse=True)
        influential_nodes = degree_ordered_nodes[:int(q*N)]
        normal_nodes = degree_ordered_nodes[int(q*N):]
        ## Simulation
        influential = simulate_spread(G, influential_nodes, phi)
        normal = simulate_spread(G, normal_nodes, phi)
        ## Store results
        S_influential[i,j] += np.mean(influential)
        S_normal[i,j] += (np.mean(normal))

100%|██████████| 50/50 [00:18<00:00,  2.67it/s]

CPU times: user 16.2 s, sys: 197 ms, total: 16.4 s
Wall time: 18.7 s


In [7]:
%%time
#################################################################################
################################## Simulation ###################################
#################################################################################

N = 25
q = 0.1   
phi = 0.18
max_n_avg = 6
increment = 0.2
num_simulations = 50
label = 'is_influenced'

n_avg = np.arange(1, max_n_avg, increment)
p = [avg/(N-1) for avg in n_avg]
n = len(p)
S_influential, S_normal = np.zeros([num_simulations, n]), np.zeros([num_simulations, n])

for i in tqdm(range(num_simulations)):
    for j in range(n):
        probability = p[j]
        G = nx.erdos_renyi_graph(N, probability)
        set_influence(G, 0)
        ## Retrieve influential nodes - top q% and non-influential nodes
        degree_ordered_nodes = sorted(list(G.nodes()), key=lambda x: G.degree(x), reverse=True)
        influential_nodes = degree_ordered_nodes[:int(q*N)]
        normal_nodes = degree_ordered_nodes[int(q*N):]
        ## Simulation
        influential = simulate_spread(G, influential_nodes, phi)
        normal = simulate_spread(G, normal_nodes, phi)
        ## Store results
        S_influential[i,j] += np.mean(influential)
        S_normal[i,j] += (np.mean(normal))

100%|██████████| 50/50 [00:18<00:00,  2.72it/s]

CPU times: user 16.5 s, sys: 307 ms, total: 16.8 s
Wall time: 18.4 s


In [45]:
def RG_iteration(N, probability):
    G = nx.erdos_renyi_graph(N, probability)
    set_influence(G, 0)
    ## Retrieve influential nodes - top q% and non-influential nodes
    degree_ordered_nodes = sorted(list(G.nodes()), key=lambda x: G.degree(x), reverse=True)
    influential_nodes = degree_ordered_nodes[:int(q*N)]
    normal_nodes = degree_ordered_nodes[int(q*N):]
    ## Simulation
    influential = delayed(simulate_spread)(G, influential_nodes, phi)
    normal = simulate_spread(G, normal_nodes, phi)
    ## Store results
    S_influential = np.mean(influential)
    S_normal = np.mean(normal)
    
    y = (S_influential, S_normal)
    return y

In [46]:
%%time
#################################################################################
################################## Simulation ###################################
#################################################################################

N = 25
q = 0.1   
phi = 0.18
max_n_avg = 6
increment = 0.2
num_simulations = 50
label = 'is_influenced'

n_avg = np.arange(1, max_n_avg, increment)
p = [avg/(N-1) for avg in n_avg]
n = len(p)
S = np.zeros([n], dtype=object)

for j in range(n):
    probability = p[j]
    results = delayed(RG_iteration)(N, probability)
    ## Store results
    S[j] = results


CPU times: user 1.5 ms, sys: 166 µs, total: 1.66 ms
Wall time: 1.81 ms


In [7]:
import dask.array as da

In [48]:
v= dask.compute(S.tolist())

In [23]:
da.compute(S[0])

((Delayed('mean-68a22ed5-3754-4069-814b-68c85c9b44fb'), 1.7391304347826086),)

In [50]:
v[0][0]

(Delayed('mean-0c6387c6-41d0-44b0-a6a0-2c49f94bd6d9'), 2.0869565217391304)

In [ ]:
##########################
### Processing Results ###
##########################
names = ["Influential", "Normal"]

# Number of Nodes of Network Influenced
N_influential = np.apply_along_axis(np.mean, 0, S_influential)
N_normal = np.apply_along_axis(np.mean, 0, S_normal)

# Proportion of Network Influenced
P_influential = [x/N for x in N_influential]
P_normal = [x/N for x in N_normal]

In [ ]:
############
### Pots ###
############

plt.plot(n_avg, N_influential, "-s")
plt.plot(n_avg, N_normal, "-o")
plt.ylabel("Average Number Influenced")
plt.xlabel("Average Degree")
plt.title("Number of Nodes Influenced")
plt.legend(names)

In [ ]:
plt.plot(n_avg[:16], P_influential[:16], "-o")
plt.plot(n_avg[:16], P_normal[:16], "-o")
plt.ylabel("Average Number Influenced")
plt.xlabel("Average Degree")
plt.title("Percentage of Network Influenced")
plt.legend(names)

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].plot(n_avg, S_influential, "-o")
axs[0].plot(n_avg, S_normal, "-o")
axs[0].set_title("Number of Nodes Influenced")
axs[0].set_ylabel("Average Number Influenced")
axs[0].set_xlabel("Average Degree")
axs[0].legend(names)

axs[1].plot(n_avg, P_influential, "-o")
axs[1].plot(n_avg, P_normal, "-o")
axs[1].set_title("Percentage of Network Influenced")
axs[1].set_ylabel("Average Number Influenced")
axs[1].set_xlabel("Average Degree")
axs[1].legend(names)

In [3]:
cpu_count()

4